- 교전하면서 회복량
    - 잃은 체력
    - 무조건 회복하는게 중요하는게 아니라 잃은 체력에 대한 회복이 중요
- 투척무기 : 연막탄, 수류탄
    - 획득 대비 못쓰는 선수들
    - over 파밍

- pubg 결과들
- telemetry를 확인해서 볼 수 있는 결과들
- 어떤 총기를 많이 사용했는지
- 총기 숙련도
- 교전 상황을 또 정의 -> 그 안에 위의 지표들 추출


- ANOVA 돌려보기

- MATCH LIST 정리

## 1. library 

In [99]:
from pprint import pprint
from collections import deque, defaultdict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from warnings import filterwarnings
filterwarnings('ignore')

## 2. data load

### 2.1. 교전 data load

In [2]:
type1_df = pd.read_pickle('./data/fight1/type1_df.pkl')
type1_df.head()

,created_at,killer,victim,fight,attacker,defender,DiK
0,2020-09-10T08:57:06Z,PG_LexGod,AZG_Untop,created_at d...,AZG_Untop,PG_LexGod,True
1,2020-09-10T08:57:06Z,ELR_YUN,GZ_Zed,created_at d...,ELR_YUN,GZ_Zed,False
2,2020-09-10T08:57:06Z,GZ_Sico,AFK_Asag2,created_at d...,GZ_Sico,AFK_Asag2,False
3,2020-09-10T08:57:06Z,STA_Jungsae,GZ_Sico,created_at d...,STA_Jungsae,GZ_Sico,False
4,2020-09-10T08:57:06Z,LW_Hari,STA_Tae9,created_at d...,LW_Hari,STA_Tae9,False


In [3]:
type2_df = pd.read_pickle('./data/fight1/type2_df.pkl')
type2_df.head()

,created_at,attacker,defender,fight
0,2020-09-11T11:07:54Z,GG_Gramatic,ELR_YUN,created_at date...
1,2020-09-11T11:07:54Z,GXG_Chaebin,FPC_Americano,created_at dat...
2,2020-09-11T11:07:54Z,NCT_karlie,BJ_Black9,created_at date...
3,2020-09-11T11:07:54Z,FPC_JIPYO,GXG_Vir,created_at dat...
4,2020-09-11T11:07:54Z,AFS_Daehwi,FPC_Cross,created_at dat...


### 2.2 참가 player load

In [4]:
player_stats = pd.read_csv('./data/fight1/player_join.csv')
player_stats

,created_at,player,death_type,time_survived(s),win
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False
...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False


## 3. 지표 정의

### 3.0. 선 포함 데이터
- plyaer_stats
|  변수명    |  설명    |
| ---- | ---- | 
|  created_at    | match time     | 
|  player    | player name     | 
|  death_type    | 사망 유형     | 
|  time_survived(s)	    | 생존 시간     | 
|  win    | 승리 여부     | 

- type1_df
|  변수명    |  설명    |
| ---- | ---- | 
|  created_at    | match time     | 
|  killer    | killer name     | 
|  victim    | victim name     | 
|  fight    | 교전 dataframe     | 
|  attacker    | attacker name     | 
|  defender    | defender name     | 
|  DiK    | Defender is Killer     | 

- type2_df
|  변수명    |  설명    |
| ---- | ---- | 
|  created_at    | match time     | 
|  attacker    | attacker name     | 
|  defender    | defender name     | 
|  fight    | 교전 dataframe     | 


### 3.1. kill수
- 사용 : type1_df 

#### 3.1.1. 총 kill

In [5]:
kill_count = type1_df.groupby(['created_at', 'killer'])['victim'].count().to_frame().reset_index()
kill_count.columns = ['created_at', 'player', 'kill']
kill_count

,created_at,player,kill
0,2020-09-10T08:57:06Z,AFK_APXB,1
1,2020-09-10T08:57:06Z,AFS_BBABBO,1
2,2020-09-10T08:57:06Z,AFS_blackwalk,1
3,2020-09-10T08:57:06Z,AZG_Rage,1
4,2020-09-10T08:57:06Z,BJ_AKaN,1
...,...,...,...
1251,2020-10-10T12:06:16Z,SP_KIMGONI,1
1252,2020-10-10T12:06:16Z,SP_NEFiEX,1
1253,2020-10-10T12:06:16Z,T1_Adder,2
1254,2020-10-10T12:06:16Z,T1_Akad,2


In [6]:
player_stats = player_stats.merge(kill_count, how='left', on=['created_at', 'player'])
player_stats['kill'] = player_stats['kill'].fillna(0)
player_stats

,created_at,player,death_type,time_survived(s),win,kill
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0
...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0


#### 3.1.2. attacker kill

In [7]:
attacker_kill_count = type1_df.query("DiK==False").groupby(['created_at', 'killer'])['victim'].count().to_frame().reset_index()
attacker_kill_count.columns = ['created_at', 'player', 'attacker_kill']
attacker_kill_count

,created_at,player,attacker_kill
0,2020-09-10T08:57:06Z,AFK_APXB,1
1,2020-09-10T08:57:06Z,AFS_BBABBO,1
2,2020-09-10T08:57:06Z,AFS_blackwalk,1
3,2020-09-10T08:57:06Z,AZG_Rage,1
4,2020-09-10T08:57:06Z,BJ_AKaN,1
...,...,...,...
1173,2020-10-10T12:06:16Z,SP_KIMGONI,1
1174,2020-10-10T12:06:16Z,SP_NEFiEX,1
1175,2020-10-10T12:06:16Z,T1_Adder,1
1176,2020-10-10T12:06:16Z,T1_Akad,2


In [8]:
player_stats = player_stats.merge(attacker_kill_count, how='left', on=['created_at', 'player'])
player_stats['attacker_kill'] = player_stats['attacker_kill'].fillna(0)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0
...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0


#### 3.1.3. defender kill

In [9]:
defender_kill_count = type1_df.query("DiK==True").groupby(['created_at', 'killer'])['victim'].count().to_frame().reset_index()
defender_kill_count.columns = ['created_at', 'player', 'defender_kill']
defender_kill_count

,created_at,player,defender_kill
0,2020-09-10T08:57:06Z,BJ_Black9,1
1,2020-09-10T08:57:06Z,GE_Laura,1
2,2020-09-10T08:57:06Z,GG_Gramatic,1
3,2020-09-10T08:57:06Z,GZ_DParis,1
4,2020-09-10T08:57:06Z,LW_JG,1
...,...,...,...
291,2020-10-10T12:06:16Z,AF_Hansia,2
292,2020-10-10T12:06:16Z,AF_daengchae,2
293,2020-10-10T12:06:16Z,GEN_Aqua5,1
294,2020-10-10T12:06:16Z,GEN_Loki,1


In [10]:
player_stats = player_stats.merge(defender_kill_count, how='left', on=['created_at', 'player'])
player_stats['defender_kill'] = player_stats['defender_kill'].fillna(0)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0


### 3.2. deal량
- 계산 : 경기당 총 deal량, attacker_deal, defender_deal
- 사용 : type1_df, type2_df 
- 세부사항 : Damage_Gun 이외 데미지 포함

In [11]:
def damage_compute(attacker, defender, fight):
    tmp = fight.query("event_type=='LogPlayerTakeDamage'").groupby(['attacker_name'])['damage'].sum()
    attacker_damage = tmp[attacker]
    if tmp.size==1:
        defender_damage = 0
    else:
        defender_damage = tmp[defender]
    return attacker_damage, defender_damage

In [12]:
type1_df[['attacker_damage', 'defender_damage']] = type1_df.apply(lambda row: damage_compute(row['attacker'], row['defender'], row['fight']), axis=1, result_type ='expand')
type1_df.head()

,created_at,killer,victim,fight,attacker,defender,DiK,attacker_damage,defender_damage
0,2020-09-10T08:57:06Z,PG_LexGod,AZG_Untop,created_at d...,AZG_Untop,PG_LexGod,True,64.357788,94.000000
1,2020-09-10T08:57:06Z,ELR_YUN,GZ_Zed,created_at d...,ELR_YUN,GZ_Zed,False,63.999865,0.000000
2,2020-09-10T08:57:06Z,GZ_Sico,AFK_Asag2,created_at d...,GZ_Sico,AFK_Asag2,False,74.599991,0.000000
3,2020-09-10T08:57:06Z,STA_Jungsae,GZ_Sico,created_at d...,STA_Jungsae,GZ_Sico,False,100.000002,30.360001
4,2020-09-10T08:57:06Z,LW_Hari,STA_Tae9,created_at d...,LW_Hari,STA_Tae9,False,53.399971,0.000000


In [13]:
type2_df[['attacker_damage', 'defender_damage']] = type2_df.apply(lambda row: damage_compute(row['attacker'], row['defender'], row['fight']), axis=1, result_type ='expand')
type2_df.head()

,created_at,attacker,defender,fight,attacker_damage,defender_damage
0,2020-09-11T11:07:54Z,GG_Gramatic,ELR_YUN,created_at date...,52.440001,80.039999
1,2020-09-11T11:07:54Z,GXG_Chaebin,FPC_Americano,created_at dat...,112.672979,0.000000
2,2020-09-11T11:07:54Z,NCT_karlie,BJ_Black9,created_at date...,97.387632,0.000000
3,2020-09-11T11:07:54Z,FPC_JIPYO,GXG_Vir,created_at dat...,0.000000,0.000000
4,2020-09-11T11:07:54Z,AFS_Daehwi,FPC_Cross,created_at dat...,29.509483,0.000000


#### 3.2.1. 총 deal

In [14]:
## type1
type1_attacker_damage = type1_df[['created_at', 'attacker', 'attacker_damage']]
type1_attacker_damage.columns = ['created_at', 'player', 'type1_damage']

type1_defender_damage = type1_df[['created_at', 'defender', 'defender_damage']]
type1_defender_damage.columns = ['created_at', 'player', 'type1_damage']

type1_group = pd.concat([type1_attacker_damage, 
                         type1_defender_damage]).groupby(['created_at', 'player'])
type1_damage = type1_group.sum().reset_index()


## typ2
type2_attacker_damage = type2_df[['created_at', 'attacker', 'attacker_damage']]
type2_attacker_damage.columns = ['created_at', 'player', 'type2_damage']

type2_defender_damage = type2_df[['created_at', 'defender', 'defender_damage']]
type2_defender_damage.columns = ['created_at', 'player', 'type2_damage']

type2_group = pd.concat([type2_attacker_damage, 
                         type2_defender_damage]).groupby(['created_at', 'player'])
type2_damage = type2_group.sum().reset_index()

In [15]:
player_stats = player_stats.merge(type1_damage, how='left', on=['created_at', 'player'])
player_stats['type1_damage'] = player_stats['type1_damage'].fillna(0)

player_stats = player_stats.merge(type2_damage, how='left', on=['created_at', 'player'])
player_stats['type2_damage'] = player_stats['type2_damage'].fillna(0)


In [16]:
player_stats['damage'] = player_stats['type1_damage'] + player_stats['type2_damage']

#### 3.2.2. attacker_deal

In [17]:
type1_attacker_damage.columns = ['created_at', 'player', 'attacker_damage'] 
type2_attacker_damage.columns = ['created_at', 'player', 'attacker_damage']
attacker_damage = pd.concat([type1_attacker_damage,
                             type2_attacker_damage]).groupby(['created_at', 'player']).sum().reset_index()
attacker_damage

,created_at,player,attacker_damage
0,2020-09-10T08:57:06Z,AFK_APXB,100.000000
1,2020-09-10T08:57:06Z,AFS_BBABBO,170.797829
2,2020-09-10T08:57:06Z,AFS_blackwalk,83.000000
3,2020-09-10T08:57:06Z,AZG_Rage,183.939083
4,2020-09-10T08:57:06Z,AZG_SexyChan,100.000000
...,...,...,...
1869,2020-10-10T12:06:16Z,VLG_Hikari,99.999998
1870,2020-10-10T12:06:16Z,VLG_Lambu,42.393847
1871,2020-10-10T12:06:16Z,VLG_Spear,81.779997
1872,2020-10-10T12:06:16Z,VSG_Foxy,33.544075


In [18]:
player_stats = player_stats.merge(attacker_damage, how='left', on=['created_at', 'player'])
player_stats['attacker_damage'] = player_stats.attacker_damage.fillna(0)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,damage,attacker_damage
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,186.056967,186.056967
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,111.782974,0.000000
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,185.693186,185.693186
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,297.907040,134.706532
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,191.487841,191.487841
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,179.347429,179.347429
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,142.227491,142.227491


#### 3.2.3. defender_deal

In [19]:
type1_defender_damage.columns = ['created_at', 'player', 'defender_damage'] 
type2_defender_damage.columns = ['created_at', 'player', 'defender_damage']
defender_damage = pd.concat([type1_defender_damage,
                             type2_defender_damage]).groupby(['created_at', 'player']).sum().reset_index()
defender_damage

,created_at,player,defender_damage
0,2020-09-10T08:57:06Z,AFK_APXB,0.000000
1,2020-09-10T08:57:06Z,AFK_Arden,0.000000
2,2020-09-10T08:57:06Z,AFK_Asag2,0.000000
3,2020-09-10T08:57:06Z,AFK_Caley,0.000000
4,2020-09-10T08:57:06Z,AFS_BBABBO,0.000000
...,...,...,...
2397,2020-10-10T12:06:16Z,VSG_WICK2D,41.428223
2398,2020-10-10T12:06:16Z,emT_Gyuyeon,0.000000
2399,2020-10-10T12:06:16Z,emT_Hwarang,77.280001
2400,2020-10-10T12:06:16Z,emT_Piky,0.000000


In [20]:
player_stats = player_stats.merge(defender_damage, how='left', on=['created_at', 'player'])
player_stats['defender_damage'] = player_stats.defender_damage.fillna(0)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,damage,attacker_damage,defender_damage
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,186.056967,186.056967,0.000000
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,111.782974,0.000000,111.782974
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,185.693186,185.693186,0.000000
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,297.907040,134.706532,163.200508
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,191.487841,191.487841,0.000000
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,179.347429,179.347429,0.000000
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,142.227491,142.227491,0.000000


### 3.3. 승률 
- 계산 : 교전 승률, attacker 승률, defender 승률
- 사용 : type1_df

#### 3.3.1. 교전 승률
- 공식 : 승리 / (승리+패배) x 100
- 세부사항
    - 생존하고 킬이 있으면 승률 : 1
    - 킬이 0이면 승률 0:
    - player에 의한 사망이면 승률 : 공식
    - 그 외(kill이 1이상이고 player에 의해 사망하지 않은 경우) : 1

In [21]:
def compute_win_rate(df):
    
    if df['death_type']=='alive' and df['kill']>0:
        return 1
    
    elif df['kill']==0:
        return 0
    
    elif df['death_type']=='byplayer':
        return df['kill']/(df['kill']+1)
    
    else:
        return 1

In [22]:
player_stats['win_rate'] = player_stats.apply(compute_win_rate, axis=1)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,damage,attacker_damage,defender_damage,win_rate
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,186.056967,186.056967,0.000000,0.000000
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,111.782974,0.000000,111.782974,0.666667
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,185.693186,185.693186,0.000000,0.500000
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,297.907040,134.706532,163.200508,0.500000
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,191.487841,191.487841,0.000000,0.000000
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,179.347429,179.347429,0.000000,0.800000
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,142.227491,142.227491,0.000000,0.000000


#### 3.3.2. attacker 승률
- 공식 : type1의 먼저 공격했을 시 승리/type1의 먼저 공격했을 시
- 변수 : {AiK : Attacker is Killer}

- 분자
    - type1_df.query('DiK==False')[['created_at', 'attacker', 'DiK']] : attacker가 이긴 경우

In [23]:
type1_df_AiK = type1_df.query('DiK==False')[['created_at', 'attacker', 'DiK']].groupby(['created_at', 'attacker']).count().reset_index()
type1_df_AiK.columns = ['created_at', 'player', 'AiK']
type1_df_AiK

,created_at,player,AiK
0,2020-09-10T08:57:06Z,AFK_APXB,1
1,2020-09-10T08:57:06Z,AFS_BBABBO,1
2,2020-09-10T08:57:06Z,AFS_blackwalk,1
3,2020-09-10T08:57:06Z,AZG_Rage,1
4,2020-09-10T08:57:06Z,BJ_AKaN,1
...,...,...,...
1173,2020-10-10T12:06:16Z,SP_KIMGONI,1
1174,2020-10-10T12:06:16Z,SP_NEFiEX,1
1175,2020-10-10T12:06:16Z,T1_Adder,1
1176,2020-10-10T12:06:16Z,T1_Akad,2


- 분모
    -  type1_df[['created_at', 'attacker', 'DiK']] : attacker인 경우

In [24]:
type1_df_attacker = type1_df[['created_at', 'attacker', 'DiK']].groupby(['created_at', 'attacker']).count().reset_index()
type1_df_attacker.columns = ['created_at', 'player', 'attack']
type1_df_attacker

,created_at,player,attack
0,2020-09-10T08:57:06Z,AFK_APXB,1
1,2020-09-10T08:57:06Z,AFS_BBABBO,1
2,2020-09-10T08:57:06Z,AFS_blackwalk,1
3,2020-09-10T08:57:06Z,AZG_Rage,1
4,2020-09-10T08:57:06Z,AZG_Untop,1
...,...,...,...
1356,2020-10-10T12:06:16Z,T1_Akad,2
1357,2020-10-10T12:06:16Z,VLG_DAEVA,1
1358,2020-10-10T12:06:16Z,VLG_Hikari,1
1359,2020-10-10T12:06:16Z,VLG_Spear,1


In [25]:
type1_attacker_win_rate = type1_df_attacker.merge(type1_df_AiK, how='left', on=['created_at', 'player'])
type1_attacker_win_rate = type1_attacker_win_rate.fillna(0)
type1_attacker_win_rate['attacker_win_rate'] = type1_attacker_win_rate['AiK'] / type1_attacker_win_rate['attack']

In [26]:
player_stats = player_stats.merge(type1_attacker_win_rate[['created_at', 'player', 'attacker_win_rate']], how='left', on=['created_at', 'player'])
player_stats['attacker_win_rate'] = player_stats['attacker_win_rate'].fillna(0)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,damage,attacker_damage,defender_damage,win_rate,attacker_win_rate
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,186.056967,186.056967,0.000000,0.000000,0.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,111.782974,0.000000,111.782974,0.666667,1.0
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,185.693186,185.693186,0.000000,0.500000,1.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,297.907040,134.706532,163.200508,0.500000,1.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,191.487841,191.487841,0.000000,0.000000,0.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,179.347429,179.347429,0.000000,0.800000,1.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,142.227491,142.227491,0.000000,0.000000,0.0


#### 3.3.3. defender 승률
- 공식 : type1의 먼저 공격받고 승리/type1의 먼저 공격받았을 때

- 분자
    - type1_df.query('DiK==True')[['created_at', 'attacker', 'DiK']] : defender가 이긴 경우

In [27]:
type1_df_DiK = type1_df.query('DiK==True')[['created_at', 'defender', 'DiK']].groupby(['created_at', 'defender']).count().reset_index()
type1_df_DiK.columns = ['created_at', 'player', 'DiK']
type1_df_DiK

,created_at,player,DiK
0,2020-09-10T08:57:06Z,BJ_Black9,1
1,2020-09-10T08:57:06Z,GE_Laura,1
2,2020-09-10T08:57:06Z,GG_Gramatic,1
3,2020-09-10T08:57:06Z,GZ_DParis,1
4,2020-09-10T08:57:06Z,LW_JG,1
...,...,...,...
291,2020-10-10T12:06:16Z,AF_Hansia,2
292,2020-10-10T12:06:16Z,AF_daengchae,2
293,2020-10-10T12:06:16Z,GEN_Aqua5,1
294,2020-10-10T12:06:16Z,GEN_Loki,1


- 분모
    - type1_df[['created_at', 'defender', 'DiK']] : defender인 경우

In [28]:
type1_df_defender = type1_df[['created_at', 'defender', 'DiK']].groupby(['created_at', 'defender']).count().reset_index()
type1_df_defender.columns = ['created_at', 'player', 'defense']

In [29]:
type1_defender_win_rate = type1_df_defender.merge(type1_df_DiK, how='left', on=['created_at', 'player'])
type1_defender_win_rate = type1_defender_win_rate.fillna(0)
type1_defender_win_rate['defender_win_rate'] = type1_defender_win_rate['DiK'] / type1_defender_win_rate['defense']

In [30]:
player_stats = player_stats.merge(type1_defender_win_rate[['created_at', 'player', 'defender_win_rate']], how='left', on=['created_at', 'player'])
player_stats['defender_win_rate'] = player_stats['defender_win_rate'].fillna(0)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,damage,attacker_damage,defender_damage,win_rate,attacker_win_rate,defender_win_rate
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,186.056967,186.056967,0.000000,0.000000,0.0,0.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,111.782974,0.000000,111.782974,0.666667,1.0,0.5
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,185.693186,185.693186,0.000000,0.500000,1.0,0.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,297.907040,134.706532,163.200508,0.500000,1.0,0.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,191.487841,191.487841,0.000000,0.000000,0.0,0.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,179.347429,179.347429,0.000000,0.800000,1.0,0.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,142.227491,142.227491,0.000000,0.000000,0.0,0.0


### 3.4.생존율
- 계산 : 교전 생존율, attacker 생존율, defender 생존율
- 사용 : type1_df, type2_df


#### 3.4.1. 교전 생존율
- 공식 : (교전1이김+교전2)/교전수

In [31]:
type1_count = type1_group.count().reset_index()
type1_count.columns = ['created_at', 'player', 'n_of_type1']
type1_count

,created_at,player,n_of_type1
0,2020-09-10T08:57:06Z,AFK_APXB,2
1,2020-09-10T08:57:06Z,AFK_Arden,1
2,2020-09-10T08:57:06Z,AFK_Asag2,1
3,2020-09-10T08:57:06Z,AFK_Caley,1
4,2020-09-10T08:57:06Z,AFS_BBABBO,2
...,...,...,...
2515,2020-10-10T12:06:16Z,VSG_WICK2D,1
2516,2020-10-10T12:06:16Z,emT_Gyuyeon,1
2517,2020-10-10T12:06:16Z,emT_Hwarang,1
2518,2020-10-10T12:06:16Z,emT_Piky,1


In [32]:
type2_count = type2_group.count().reset_index()
type2_count.columns = ['created_at', 'player', 'n_of_type2']
type2_count

,created_at,player,n_of_type2
0,2020-09-10T08:57:06Z,AFK_Arden,2
1,2020-09-10T08:57:06Z,AFK_Asag2,1
2,2020-09-10T08:57:06Z,AFS_BBABBO,3
3,2020-09-10T08:57:06Z,AFS_Daehwi,1
4,2020-09-10T08:57:06Z,AFS_Jeonha,1
...,...,...,...
2111,2020-10-10T12:06:16Z,VSG_WICK2D,1
2112,2020-10-10T12:06:16Z,emT_Gyuyeon,1
2113,2020-10-10T12:06:16Z,emT_Hwarang,2
2114,2020-10-10T12:06:16Z,emT_Piky,1


In [33]:
player_stats = player_stats.merge(type1_count, how='left', on=['created_at', 'player'])
player_stats['n_of_type1'] = player_stats['n_of_type1'].fillna(0)

player_stats = player_stats.merge(type2_count, how='left', on=['created_at', 'player'])
player_stats['n_of_type2'] = player_stats['n_of_type2'].fillna(0)


In [34]:
player_stats['survival_rate'] = (player_stats.kill + player_stats.n_of_type2)/(player_stats.n_of_type1 + player_stats.n_of_type2)
player_stats['survival_rate'] = player_stats.survival_rate.fillna(0)

#### 3.4.2. attacker 생존율
- 공식 : (교전1 attacker 이김+교전2 attacker)/(attacker 교전수)

In [35]:
type2_df_attacker = type2_df[['created_at', 'attacker', 'fight']].groupby(['created_at', 'attacker']).count().reset_index()
type2_df_attacker.columns = ['created_at', 'player', 'type2_num_of_attack']
type2_df_attacker

,created_at,player,type2_num_of_attack
0,2020-09-10T08:57:06Z,AFS_BBABBO,2
1,2020-09-10T08:57:06Z,AFS_blackwalk,1
2,2020-09-10T08:57:06Z,AZG_Rage,2
3,2020-09-10T08:57:06Z,AZG_SexyChan,1
4,2020-09-10T08:57:06Z,BJ_AKaN,1
...,...,...,...
1444,2020-10-10T12:06:16Z,SP_NEFiEX,1
1445,2020-10-10T12:06:16Z,SP_maczora,1
1446,2020-10-10T12:06:16Z,VLG_Lambu,1
1447,2020-10-10T12:06:16Z,VLG_Spear,1


In [36]:
type1_df_attacker.columns = ['created_at', 'player', 'type1_num_of_attack']

In [37]:
player_stats = player_stats.merge(type1_df_attacker, how='left', on=['created_at', 'player'])
player_stats = player_stats.merge(type2_df_attacker, how='left', on=['created_at', 'player'])
player_stats.type1_num_of_attack.fillna(0, inplace=True)
player_stats.type2_num_of_attack.fillna(0, inplace=True)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,attacker_damage,defender_damage,win_rate,attacker_win_rate,defender_win_rate,n_of_type1,n_of_type2,survival_rate,type1_num_of_attack,type2_num_of_attack
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,186.056967,0.000000,0.000000,0.0,0.0,1.0,4.0,0.800000,1.0,1.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,1.0,1.0,0.500000,0.0,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,0.000000,111.782974,0.666667,1.0,0.5,3.0,2.0,0.800000,1.0,0.0
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,185.693186,0.000000,0.500000,1.0,0.0,2.0,3.0,0.800000,1.0,1.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,1.0,1.0,0.500000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,134.706532,163.200508,0.500000,1.0,0.0,2.0,5.0,0.857143,1.0,2.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,191.487841,0.000000,0.000000,0.0,0.0,0.0,5.0,1.000000,0.0,4.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,179.347429,0.000000,0.800000,1.0,0.0,5.0,3.0,0.875000,4.0,1.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,142.227491,0.000000,0.000000,0.0,0.0,1.0,4.0,0.800000,0.0,3.0


In [38]:
player_stats['attacker_survival_rate']= (player_stats['attacker_kill'] + player_stats['type2_num_of_attack']) / (player_stats['type1_num_of_attack'] + player_stats['type2_num_of_attack'])
player_stats.attacker_survival_rate.fillna(0, inplace=True)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,defender_damage,win_rate,attacker_win_rate,defender_win_rate,n_of_type1,n_of_type2,survival_rate,type1_num_of_attack,type2_num_of_attack,attacker_survival_rate
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,0.000000,0.000000,0.0,0.0,1.0,4.0,0.800000,1.0,1.0,0.5
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,1.0,1.0,0.500000,0.0,0.0,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,111.782974,0.666667,1.0,0.5,3.0,2.0,0.800000,1.0,0.0,1.0
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,0.000000,0.500000,1.0,0.0,2.0,3.0,0.800000,1.0,1.0,1.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,1.0,1.0,0.500000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,163.200508,0.500000,1.0,0.0,2.0,5.0,0.857143,1.0,2.0,1.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,0.000000,0.000000,0.0,0.0,0.0,5.0,1.000000,0.0,4.0,1.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,0.000000,0.800000,1.0,0.0,5.0,3.0,0.875000,4.0,1.0,1.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,0.000000,0.000000,0.0,0.0,1.0,4.0,0.800000,0.0,3.0,1.0


#### 3.4.3. defender 생존율
- 공식 : (교전1 defender 이김+교전2 defender)/(defender 교전수)

In [39]:
type2_df_defender = type2_df[['created_at', 'defender', 'fight']].groupby(['created_at', 'defender']).count().reset_index()
type2_df_defender.columns = ['created_at', 'player', 'type2_num_of_defense']
type2_df_defender

,created_at,player,type2_num_of_defense
0,2020-09-10T08:57:06Z,AFK_Arden,2
1,2020-09-10T08:57:06Z,AFK_Asag2,1
2,2020-09-10T08:57:06Z,AFS_BBABBO,1
3,2020-09-10T08:57:06Z,AFS_Daehwi,1
4,2020-09-10T08:57:06Z,AFS_Jeonha,1
...,...,...,...
1657,2020-10-10T12:06:16Z,VSG_WICK2D,1
1658,2020-10-10T12:06:16Z,emT_Gyuyeon,1
1659,2020-10-10T12:06:16Z,emT_Hwarang,1
1660,2020-10-10T12:06:16Z,emT_Piky,1


In [40]:
type1_df_defender.columns =  ['created_at', 'player', 'type1_num_of_defense']

In [41]:
player_stats = player_stats.merge(type1_df_defender, how='left', on=['created_at', 'player'])
player_stats = player_stats.merge(type2_df_defender, how='left', on=['created_at', 'player'])
player_stats.type1_num_of_defense.fillna(0, inplace=True)
player_stats.type2_num_of_defense.fillna(0, inplace=True)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,attacker_win_rate,defender_win_rate,n_of_type1,n_of_type2,survival_rate,type1_num_of_attack,type2_num_of_attack,attacker_survival_rate,type1_num_of_defense,type2_num_of_defense
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,0.0,0.0,1.0,4.0,0.800000,1.0,1.0,0.5,0.0,3.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,1.0,1.0,0.500000,0.0,0.0,0.0,1.0,1.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,1.0,0.5,3.0,2.0,0.800000,1.0,0.0,1.0,2.0,2.0
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,1.0,0.0,2.0,3.0,0.800000,1.0,1.0,1.0,1.0,2.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,1.0,1.0,0.500000,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,1.0,0.0,2.0,5.0,0.857143,1.0,2.0,1.0,1.0,3.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,0.0,0.0,0.0,5.0,1.000000,0.0,4.0,1.0,0.0,1.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,1.0,0.0,5.0,3.0,0.875000,4.0,1.0,1.0,1.0,2.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,0.0,0.0,1.0,4.0,0.800000,0.0,3.0,1.0,1.0,1.0


In [42]:
player_stats['defender_survival_rate']= (player_stats['defender_kill'] + player_stats['type2_num_of_defense']) / (player_stats['type1_num_of_defense'] + player_stats['type2_num_of_defense'])
player_stats.defender_survival_rate.fillna(0, inplace=True)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,defender_win_rate,n_of_type1,n_of_type2,survival_rate,type1_num_of_attack,type2_num_of_attack,attacker_survival_rate,type1_num_of_defense,type2_num_of_defense,defender_survival_rate
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,0.0,1.0,4.0,0.800000,1.0,1.0,0.5,0.0,3.0,1.000000
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,1.0,1.0,0.500000,0.0,0.0,0.0,1.0,1.0,0.500000
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,0.5,3.0,2.0,0.800000,1.0,0.0,1.0,2.0,2.0,0.750000
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,0.0,2.0,3.0,0.800000,1.0,1.0,1.0,1.0,2.0,0.666667
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,1.0,1.0,0.500000,0.0,0.0,0.0,1.0,1.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,0.0,2.0,5.0,0.857143,1.0,2.0,1.0,1.0,3.0,0.750000
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,0.0,0.0,5.0,1.000000,0.0,4.0,1.0,0.0,1.0,1.000000
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,0.0,5.0,3.0,0.875000,4.0,1.0,1.0,1.0,2.0,0.666667
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,0.0,1.0,4.0,0.800000,0.0,3.0,1.0,1.0,1.0,0.500000


### 3.5. 명중률

- 경우 : 교전, attacker, defender

- 사용 : type1_df, type2_df

- 추출 데이터
    - LogPlayerTakeDamage → damage_type_category = Damage_Gun
    - LogPlayerAttack →	weapon_category	= Weapon 
    
- 공식 
    - 1. 시작 log가 take_damage(Damage_Gun)로 시작 한 경우 : take_damage 횟수/ (attack 횟수 + 1)
    - 2. 나머지 공격 의 경우 : take_damage 횟수/ attack 횟수
    - 3. 총으로 한발도 안 쏜 경우 : NaN


In [43]:
def compute_type1_acc(df):
    fight = df['fight']
    tmp = fight.query("(event_type=='LogPlayerTakeDamage' & damage_type_category=='Damage_Gun' ) | (event_type=='LogPlayerAttack' & weapon_category=='Weapon')") 
    attacker, defender = fight.iloc[0,:]['attacker_name'], fight.iloc[0,:]['defender_name']
    killer, victim = fight.iloc[-1,:]['attacker_name'], fight.iloc[-1,:]['defender_name']
    
    summary = tmp.groupby(['event_type', 'attacker_name'])['date'].count().reset_index()
    summary.columns = ['event_type', 'attacker_name', 'cnt']
    
    
    if attacker!=killer:
        # 선공(victim)
        if not summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'").empty: 
            if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'").empty:
                if fight.iloc[0,:]['damage_type_category']=='Damage_Gun':
                    v_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'")['cnt'].values[0]/
                            (summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'")['cnt'].values[0]+1)),1)
                else:
                    v_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'")['cnt'].values[0]/
                            (summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'")['cnt'].values[0])), 1)
            else:
                v_acc = 0
        else: # 공격로그가 없다면
            if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'").empty:
                v_acc = 1 
            else:
                v_acc = 0 # np.nan
                
        ## 후공(killer)
        if not summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{defender}'").empty: # 공격 로그 있고
            if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{defender}'").empty: # 데미지 로그 있으면
                k_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{defender}'")['cnt'].values[0]/
                         summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{defender}'")['cnt'].values[0]),1)
            else: # 데미지 로그가 결측이면
                k_acc = 1
        else: # 공격 로그가 없다면
            k_acc = 1

    else: # 선공이 killer이면
        if not summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'").empty: # 공격로그 있고
            if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'").empty: # 데미지 로그 있으면
                if fight.iloc[0,:]['damage_type_category']=='Damage_Gun':
                    k_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'")['cnt'].values[0]/
                             (summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'")['cnt'].values[0]+1)),1)
                else:
                    k_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'")['cnt'].values[0]/
                             (summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'")['cnt'].values[0])),1)
            else:
                k_acc = 0
        else: # 공격 로그가 없다면 
            if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'").empty:
                k_acc = 1
            else:
                k_acc = 0 #np.nan

        ## 후공(victim)
        if not summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{defender}'").empty: # 공격 로그 있고
            if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{defender}'").empty: # 데미지 로그 있으면
                v_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{defender}'")['cnt'].values[0]/
                         summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{defender}'")['cnt'].values[0]),1)
            else: # 공격은 했지만 데미지는 못 준 경우
                v_acc = 0 
        else: # 공격로그가 없을 때
            if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{defender}'").empty: # 데미지 로그가 있으면
                v_acc = 1 # 공격 로그 등장 오류 -> 1로 처리
            else: 
                v_acc = 0 #np.nan  # 한발도 못 쏜 경우
    return k_acc, v_acc

#-----------------------------------------------------------------------------------------------------------------------------#
def compute_type2_acc(df):
    fight = df['fight']
    tmp = fight.query("(event_type=='LogPlayerTakeDamage' & damage_type_category=='Damage_Gun' ) | (event_type=='LogPlayerAttack' & weapon_category=='Weapon')") 
    attacker, defender = fight.iloc[0,:]['attacker_name'], fight.iloc[0,:]['defender_name']
    
    summary = tmp.groupby(['event_type', 'attacker_name'])['date'].count().reset_index()
    summary.columns = ['event_type', 'attacker_name', 'cnt']
    
    
    if not summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'").empty: # 공격로그 있고
        if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'").empty: # 데미지 로그 있으면
            if fight.iloc[0,:]['damage_type_category']=='Damage_Gun':
                a_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'")['cnt'].values[0]/
                         (summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'")['cnt'].values[0]+1)),1)
            else:
                a_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'")['cnt'].values[0]/
                         (summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{attacker}'")['cnt'].values[0])),1)
        else:
            a_acc = 0
    else: # 공격 로그가 없다면 
        if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{attacker}'").empty:
            a_acc = 1
        else:
            a_acc = 0 # np.nan

    ## 후공(victim)
    if not summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{defender}'").empty: # 공격 로그 있고
        if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{defender}'").empty: # 데미지 로그 있으면
            d_acc = min((summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{defender}'")['cnt'].values[0]/
                     summary.query(f"event_type=='LogPlayerAttack' & attacker_name=='{defender}'")['cnt'].values[0]),1)
        else: # 공격은 했지만 데미지는 못 준 경우
            d_acc = 0 
    else: # 공격로그가 없을 때
        if not summary.query(f"event_type=='LogPlayerTakeDamage' & attacker_name=='{defender}'").empty: # 데미지 로그가 있으면
            d_acc = 1 # 공격 로그 등장 오류 -> 1로 처리
        else: 
            d_acc = 0 # np.nan  # 한발도 못 쏜 경우
    return a_acc, d_acc

In [45]:
type1_df[['killer_acc', 'victim_acc']] = type1_df.apply(lambda row: compute_type1_acc(row), axis=1, result_type ='expand')
type2_df[['attacker_acc', 'defender_acc']] = type2_df.apply(lambda row: compute_type2_acc(row), axis=1, result_type ='expand')

type1_df['attacker_acc'] = type1_df.apply(lambda row: row['victim_acc'] if row['DiK']==True else row['killer_acc'], axis=1)
type1_df['defender_acc'] = type1_df.apply(lambda row: row['killer_acc'] if row['DiK']==True else row['victim_acc'], axis=1)

In [46]:
# type1_killer_acc = type1_df[['created_at', 'killer', 'killer_acc']]
# type1_killer_acc.columns = ['created_at', 'player', 'acc']

# type1_victim_acc = type1_df[['created_at', 'victim', 'victim_acc']]
# type1_victim_acc.columns = ['created_at', 'player', 'acc']

type1_attacker_acc = type1_df[['created_at', 'attacker', 'attacker_acc']]
type1_attacker_acc.columns = ['created_at', 'player', 'acc']

type1_defender_acc = type1_df[['created_at', 'defender', 'defender_acc']]
type1_defender_acc.columns = ['created_at', 'player', 'acc']

type2_attacker_acc = type2_df[['created_at', 'attacker', 'attacker_acc']]
type2_attacker_acc.columns = ['created_at', 'player', 'acc']

type2_defender_acc = type2_df[['created_at', 'defender', 'defender_acc']]
type2_defender_acc.columns = ['created_at', 'player', 'acc']


#### 3.5.1. 교전 acc

In [47]:
acc_df = pd.concat([type1_attacker_acc, type1_defender_acc, 
                    type2_attacker_acc, type2_defender_acc]).groupby(['created_at', 'player']).mean().reset_index()
acc_df

,created_at,player,acc
0,2020-09-10T08:57:06Z,AFK_APXB,0.160000
1,2020-09-10T08:57:06Z,AFK_Arden,0.000000
2,2020-09-10T08:57:06Z,AFK_Asag2,0.000000
3,2020-09-10T08:57:06Z,AFK_Caley,0.000000
4,2020-09-10T08:57:06Z,AFS_BBABBO,0.280556
...,...,...,...
2549,2020-10-10T12:06:16Z,VSG_WICK2D,0.000000
2550,2020-10-10T12:06:16Z,emT_Gyuyeon,0.000000
2551,2020-10-10T12:06:16Z,emT_Hwarang,0.500000
2552,2020-10-10T12:06:16Z,emT_Piky,0.000000


In [48]:
player_stats = player_stats.merge(acc_df, how='left', on=['created_at', 'player'])
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,n_of_type1,n_of_type2,survival_rate,type1_num_of_attack,type2_num_of_attack,attacker_survival_rate,type1_num_of_defense,type2_num_of_defense,defender_survival_rate,acc
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,1.0,4.0,0.800000,1.0,1.0,0.5,0.0,3.0,1.000000,0.092571
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,1.0,1.0,0.500000,0.0,0.0,0.0,1.0,1.0,0.500000,0.000000
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,3.0,2.0,0.800000,1.0,0.0,1.0,2.0,2.0,0.750000,0.300000
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,2.0,3.0,0.800000,1.0,1.0,1.0,1.0,2.0,0.666667,0.078325
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,1.0,1.0,0.500000,0.0,0.0,0.0,1.0,1.0,0.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,2.0,5.0,0.857143,1.0,2.0,1.0,1.0,3.0,0.750000,0.116772
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,0.0,5.0,1.000000,0.0,4.0,1.0,0.0,1.0,1.000000,0.044714
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,5.0,3.0,0.875000,4.0,1.0,1.0,1.0,2.0,0.666667,0.294173
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,1.0,4.0,0.800000,0.0,3.0,1.0,1.0,1.0,0.500000,0.237003


#### 3.5.2. attacker acc

In [49]:
attacker_acc_df = pd.concat([type1_attacker_acc, type2_attacker_acc]).groupby(['created_at', 'player']).mean().reset_index()
attacker_acc_df.columns = ['created_at', 'player', 'attacker_acc']
attacker_acc_df

,created_at,player,attacker_acc
0,2020-09-10T08:57:06Z,AFK_APXB,0.320000
1,2020-09-10T08:57:06Z,AFS_BBABBO,0.467593
2,2020-09-10T08:57:06Z,AFS_blackwalk,0.589286
3,2020-09-10T08:57:06Z,AZG_Rage,0.702703
4,2020-09-10T08:57:06Z,AZG_SexyChan,1.000000
...,...,...,...
1869,2020-10-10T12:06:16Z,VLG_Hikari,0.562500
1870,2020-10-10T12:06:16Z,VLG_Lambu,0.200000
1871,2020-10-10T12:06:16Z,VLG_Spear,0.433333
1872,2020-10-10T12:06:16Z,VSG_Foxy,0.013889


In [50]:
player_stats = player_stats.merge(attacker_acc_df, how='left', on=['created_at', 'player'])

#### 3.5.3. defender acc

In [51]:
defender_acc_df = pd.concat([type1_defender_acc, type2_defender_acc]).groupby(['created_at', 'player']).mean().reset_index()
defender_acc_df.columns = ['created_at', 'player', 'defender_acc']
defender_acc_df

,created_at,player,defender_acc
0,2020-09-10T08:57:06Z,AFK_APXB,0.0
1,2020-09-10T08:57:06Z,AFK_Arden,0.0
2,2020-09-10T08:57:06Z,AFK_Asag2,0.0
3,2020-09-10T08:57:06Z,AFK_Caley,0.0
4,2020-09-10T08:57:06Z,AFS_BBABBO,0.0
...,...,...,...
2397,2020-10-10T12:06:16Z,VSG_WICK2D,0.0
2398,2020-10-10T12:06:16Z,emT_Gyuyeon,0.0
2399,2020-10-10T12:06:16Z,emT_Hwarang,0.5
2400,2020-10-10T12:06:16Z,emT_Piky,0.0


In [52]:
player_stats = player_stats.merge(defender_acc_df, how='left', on=['created_at', 'player'])

In [53]:
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,survival_rate,type1_num_of_attack,type2_num_of_attack,attacker_survival_rate,type1_num_of_defense,type2_num_of_defense,defender_survival_rate,acc,attacker_acc,defender_acc
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,0.800000,1.0,1.0,0.5,0.0,3.0,1.000000,0.092571,0.160000,0.047619
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,0.500000,0.0,0.0,0.0,1.0,1.0,0.500000,0.000000,NaN,0.000000
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,0.800000,1.0,0.0,1.0,2.0,2.0,0.750000,0.300000,0.500000,0.250000
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,0.800000,1.0,1.0,1.0,1.0,2.0,0.666667,0.078325,0.195813,0.000000
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,0.500000,0.0,0.0,0.0,1.0,1.0,0.500000,0.000000,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,0.857143,1.0,2.0,1.0,1.0,3.0,0.750000,0.116772,0.257143,0.011494
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,1.000000,0.0,4.0,1.0,0.0,1.0,1.000000,0.044714,0.055892,0.000000
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,0.875000,4.0,1.0,1.0,1.0,2.0,0.666667,0.294173,0.470677,0.000000
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,0.800000,0.0,3.0,1.0,1.0,1.0,0.500000,0.237003,0.395005,0.000000


### ~3.6. 먼저 공격 받고 생존 확률~ -> 3.5.2
- 공식 : 먼저 공격 받고 생존 횟수 / 먼저 공격 받은 수
- 변수 :
    - 결과 : {'DBS' : 'Defend But Survived'}  : 
    - type1 : {'DiK' : 'Defender is Killer'}
        - 먼저 공격 받고 생존 : True
        - 그 외 : False
    - type1, type2 : {'n_of_defender': 'the number of the position of defender for each player'}
        

In [54]:
# type1_DiK = type1_df.groupby(['created_at', 'defender'])['DiK'].sum().reset_index()
# type1_DiK.columns = ['created_at', 'player', 'DiK']
# type1_DiK

In [55]:
# type1_defender = type1_df.groupby(['created_at', 'defender'])['fight'].count().reset_index()
# type1_defender.columns= ['created_at', 'player', 'type1_n_of_defender']
# type1_defender

In [56]:
# type2_defender = type2_df.groupby(['created_at', 'defender'])['fight'].count().reset_index()
# type2_defender.columns= ['created_at', 'player', 'type2_n_of_defender']
# type2_defender

In [57]:
# player_stats = player_stats.merge(type1_DiK, how='left', on=['created_at', 'player'])
# player_stats['DiK'] = player_stats['DiK'].fillna(0)

# player_stats = player_stats.merge(type1_defender, how='left', on=['created_at', 'player'])
# player_stats['type1_n_of_defender'] = player_stats['type1_n_of_defender'].fillna(0)

# player_stats = player_stats.merge(type2_defender, how='left', on=['created_at', 'player'])
# player_stats['type2_n_of_defender'] = player_stats['type2_n_of_defender'].fillna(0)

In [58]:
# player_stats['DBS'] = (player_stats['DiK'] + player_stats['type2_n_of_defender'])/(player_stats['type1_n_of_defender'] + player_stats['type2_n_of_defender'])
# player_stats

### 3.7. type1 교전에서의 killer 사격거리(mean)
- shortest_kill, longest_kill 고려할지?

In [59]:
def type1_distance_compute(df):
    
    killer = df['killer']
    df_fight = df['fight']
    
    tmp1 = df_fight.query(f"event_type=='LogPlayerTakeDamage' & damage_type_category == 'Damage_Gun' & attacker_name=='{killer}'")
    tmp2 = df_fight.query(f"event_type=='LogPlayerKill' & damage_type_category == 'Damage_Gun' & attacker_name=='{killer}'")
    if not tmp1.empty and tmp2.empty:
        tmp1['distance'] = tmp1.apply(lambda row:  np.linalg.norm(row[['attacker_x', 'attacker_y', 'attacker_z']].values - row[['defender_x', 'defender_y', 'defender_z']].values), axis=1)
        tmp2['distance'] = tmp2.apply(lambda row:  np.linalg.norm(row[['attacker_x', 'attacker_y', 'attacker_z']].values - row[['victim_x', 'victim_y', 'victim_z']].values), axis=1)
        return pd.concat([tmp1, tmp2])['distance'].mean()
    elif not tmp1.empty:
        tmp1['distance'] = tmp1.apply(lambda row:  np.linalg.norm(row[['attacker_x', 'attacker_y', 'attacker_z']].values - row[['defender_x', 'defender_y', 'defender_z']].values), axis=1)
        return tmp1.distance.mean()
    elif not tmp2.empty:
        tmp2['distance'] = tmp2.apply(lambda row:  np.linalg.norm(row[['attacker_x', 'attacker_y', 'attacker_z']].values - row[['victim_x', 'victim_y', 'victim_z']].values), axis=1)
        return tmp2.distance.values[0]
    return 0
        

In [60]:
type1_df['type1_distance'] = type1_df.apply(lambda row: type1_distance_compute(row), axis=1 )
type1_df['type1_distance'] = type1_df.type1_distance.fillna(0)

In [61]:
type1_distance = type1_df[['created_at', 'killer', 'type1_distance']].groupby(['created_at', 'killer']).mean().reset_index()
type1_distance.columns = ['created_at', 'player', 'type1_distance']
type1_distance

,created_at,player,type1_distance
0,2020-09-10T08:57:06Z,AFK_APXB,3388.019694
1,2020-09-10T08:57:06Z,AFS_BBABBO,5316.152998
2,2020-09-10T08:57:06Z,AFS_blackwalk,20996.846721
3,2020-09-10T08:57:06Z,AZG_Rage,8012.555783
4,2020-09-10T08:57:06Z,BJ_AKaN,8504.780346
...,...,...,...
1251,2020-10-10T12:06:16Z,SP_KIMGONI,8325.857943
1252,2020-10-10T12:06:16Z,SP_NEFiEX,2122.097132
1253,2020-10-10T12:06:16Z,T1_Adder,6335.653787
1254,2020-10-10T12:06:16Z,T1_Akad,4631.413561


In [62]:
player_stats = player_stats.merge(type1_distance, how='left', on=['created_at', 'player'])

In [63]:
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,type1_num_of_attack,type2_num_of_attack,attacker_survival_rate,type1_num_of_defense,type2_num_of_defense,defender_survival_rate,acc,attacker_acc,defender_acc,type1_distance
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,1.0,1.0,0.5,0.0,3.0,1.000000,0.092571,0.160000,0.047619,NaN
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,1.0,1.0,0.500000,0.000000,NaN,0.000000,NaN
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,1.0,0.0,1.0,2.0,2.0,0.750000,0.300000,0.500000,0.250000,1512.380409
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,1.0,1.0,1.0,1.0,2.0,0.666667,0.078325,0.195813,0.000000,6971.851654
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,1.0,1.0,0.500000,0.000000,NaN,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,1.0,2.0,1.0,1.0,3.0,0.750000,0.116772,0.257143,0.011494,17597.325612
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,0.0,4.0,1.0,0.0,1.0,1.000000,0.044714,0.055892,0.000000,NaN
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,4.0,1.0,1.0,1.0,2.0,0.666667,0.294173,0.470677,0.000000,21704.925898
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,0.0,3.0,1.0,1.0,1.0,0.500000,0.237003,0.395005,0.000000,NaN


### 3.8. 교전 승리 시간

In [64]:
def compute_win_time(df):
    df_fight = df['fight']
    df_fight['date'] = pd.to_datetime(df_fight.date)
    return (df_fight.iloc[-1,:]['date'] - df_fight.iloc[0,:]['date']).seconds

In [65]:
type1_df['win_time(s)'] = type1_df.apply(lambda row: compute_win_time(row), axis=1 )
type1_df

,created_at,killer,victim,fight,attacker,defender,DiK,attacker_damage,defender_damage,killer_acc,victim_acc,attacker_acc,defender_acc,type1_distance,win_time(s)
0,2020-09-10T08:57:06Z,PG_LexGod,AZG_Untop,created_at da...,AZG_Untop,PG_LexGod,True,64.357788,94.000000,0.147059,0.035714,0.035714,0.147059,658.009415,22
1,2020-09-10T08:57:06Z,ELR_YUN,GZ_Zed,created_at da...,ELR_YUN,GZ_Zed,False,63.999865,0.000000,0.266667,0.000000,0.266667,0.000000,8918.492726,6
2,2020-09-10T08:57:06Z,GZ_Sico,AFK_Asag2,created_at da...,GZ_Sico,AFK_Asag2,False,74.599991,0.000000,0.090909,0.000000,0.090909,0.000000,18522.709653,8
3,2020-09-10T08:57:06Z,STA_Jungsae,GZ_Sico,created_at da...,STA_Jungsae,GZ_Sico,False,100.000002,30.360001,0.571429,0.166667,0.571429,0.166667,443.238558,5
4,2020-09-10T08:57:06Z,LW_Hari,STA_Tae9,created_at da...,LW_Hari,STA_Tae9,False,53.399971,0.000000,0.250000,0.000000,0.250000,0.000000,22715.432121,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,2020-10-10T12:06:16Z,AF_Hansia,DNW_LeClo,created_at dat...,DNW_LeClo,AF_Hansia,True,64.860001,78.456184,0.083333,0.076923,0.076923,0.083333,2743.971146,22
2365,2020-10-10T12:06:16Z,AF_Hansia,DNW_Draft,created_at dat...,AF_Hansia,DNW_Draft,False,100.000002,0.000000,1.000000,0.000000,1.000000,0.000000,895.959848,0
2366,2020-10-10T12:06:16Z,ENT_Seongjang,AF_R0wha,created_at dat...,ENT_Seongjang,AF_R0wha,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2367,2020-10-10T12:06:16Z,ENT_Seongjang,AF_Hansia,created_at dat...,ENT_Seongjang,AF_Hansia,False,65.179092,40.364998,0.172414,0.400000,0.172414,0.400000,2222.608463,11


In [66]:
type1_win_time = type1_df[['created_at', 'killer', 'win_time(s)']].groupby(['created_at', 'killer']).mean().reset_index()
type1_win_time.columns = ['created_at', 'player', 'win_time(s)']
type1_win_time

,created_at,player,win_time(s)
0,2020-09-10T08:57:06Z,AFK_APXB,11.0
1,2020-09-10T08:57:06Z,AFS_BBABBO,7.0
2,2020-09-10T08:57:06Z,AFS_blackwalk,2.0
3,2020-09-10T08:57:06Z,AZG_Rage,84.0
4,2020-09-10T08:57:06Z,BJ_AKaN,2.0
...,...,...,...
1251,2020-10-10T12:06:16Z,SP_KIMGONI,175.0
1252,2020-10-10T12:06:16Z,SP_NEFiEX,105.0
1253,2020-10-10T12:06:16Z,T1_Adder,136.5
1254,2020-10-10T12:06:16Z,T1_Akad,2.5


In [67]:
player_stats = player_stats.merge(type1_win_time, how='left', on=['created_at', 'player'])
player_stats['win_time(s)'] = player_stats['win_time(s)'].fillna(9999)
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,type2_num_of_attack,attacker_survival_rate,type1_num_of_defense,type2_num_of_defense,defender_survival_rate,acc,attacker_acc,defender_acc,type1_distance,win_time(s)
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,1.0,0.5,0.0,3.0,1.000000,0.092571,0.160000,0.047619,NaN,9999.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,1.0,1.0,0.500000,0.000000,NaN,0.000000,NaN,9999.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,0.0,1.0,2.0,2.0,0.750000,0.300000,0.500000,0.250000,1512.380409,8.5
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,1.0,1.0,1.0,2.0,0.666667,0.078325,0.195813,0.000000,6971.851654,3.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,1.0,1.0,0.500000,0.000000,NaN,0.000000,NaN,9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,2.0,1.0,1.0,3.0,0.750000,0.116772,0.257143,0.011494,17597.325612,4.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,4.0,1.0,0.0,1.0,1.000000,0.044714,0.055892,0.000000,NaN,9999.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,1.0,1.0,1.0,2.0,0.666667,0.294173,0.470677,0.000000,21704.925898,13.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,3.0,1.0,1.0,1.0,0.500000,0.237003,0.395005,0.000000,NaN,9999.0


### 3.9. 선공 횟수

In [68]:
player_stats['num_of_attack'] = player_stats.type1_num_of_attack + player_stats.type2_num_of_attack
player_stats

,created_at,player,death_type,time_survived(s),win,kill,attacker_kill,defender_kill,type1_damage,type2_damage,...,attacker_survival_rate,type1_num_of_defense,type2_num_of_defense,defender_survival_rate,acc,attacker_acc,defender_acc,type1_distance,win_time(s),num_of_attack
0,2020-09-25T10:28:20Z,VSG_Foxy,byplayer,1263.289,False,0.0,0.0,0.0,82.800001,103.256966,...,0.5,0.0,3.0,1.000000,0.092571,0.160000,0.047619,NaN,9999.0,2.0
1,2020-09-25T10:28:20Z,VSG_DG98,byplayer,1098.276,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,1.0,1.0,0.500000,0.000000,NaN,0.000000,NaN,9999.0,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,byplayer,1263.279,False,2.0,1.0,1.0,111.782974,0.000000,...,1.0,2.0,2.0,0.750000,0.300000,0.500000,0.250000,1512.380409,8.5,1.0
3,2020-09-25T10:28:20Z,VSG_Hulk,byplayer,1236.989,False,1.0,1.0,0.0,100.000000,85.693186,...,1.0,1.0,2.0,0.666667,0.078325,0.195813,0.000000,6971.851654,3.0,2.0
4,2020-09-25T10:28:20Z,GEN_Loki,byplayer,1655.140,False,0.0,0.0,0.0,0.000000,0.000000,...,0.0,1.0,1.0,0.500000,0.000000,NaN,0.000000,NaN,9999.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,byplayer,1700.912,False,1.0,1.0,0.0,222.384956,75.522083,...,1.0,1.0,3.0,0.750000,0.116772,0.257143,0.011494,17597.325612,4.0,3.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,byzone,1578.199,False,0.0,0.0,0.0,0.000000,191.487841,...,1.0,0.0,1.0,1.000000,0.044714,0.055892,0.000000,NaN,9999.0,4.0
2557,2020-09-26T11:42:20Z,VSG_DG98,byplayer,1578.188,False,4.0,4.0,0.0,114.541283,64.806147,...,1.0,1.0,2.0,0.666667,0.294173,0.470677,0.000000,21704.925898,13.0,5.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,byplayer,1393.822,False,0.0,0.0,0.0,0.000000,142.227491,...,1.0,1.0,1.0,0.500000,0.237003,0.395005,0.000000,NaN,9999.0,3.0


In [92]:
### 3.10.

## 4. 평균

In [70]:
player_stats.columns

Index(['created_at', 'player', 'death_type', 'time_survived(s)', 'win', 'kill',
       'attacker_kill', 'defender_kill', 'type1_damage', 'type2_damage',
       'damage', 'attacker_damage', 'defender_damage', 'win_rate',
       'attacker_win_rate', 'defender_win_rate', 'n_of_type1', 'n_of_type2',
       'survival_rate', 'type1_num_of_attack', 'type2_num_of_attack',
       'attacker_survival_rate', 'type1_num_of_defense',
       'type2_num_of_defense', 'defender_survival_rate', 'acc', 'attacker_acc',
       'defender_acc', 'type1_distance', 'win_time(s)', 'num_of_attack'],
      dtype='object')

In [80]:
player_stats1 = player_stats.copy()[['created_at', 'player', 'time_survived(s)', 'kill', 'attacker_kill', 'defender_kill', 
                                     'damage', 'attacker_damage', 'defender_damage', 'win_rate', 'attacker_win_rate', 'defender_win_rate',
                                     'survival_rate', 'attacker_survival_rate', 'defender_survival_rate',
                                     'acc', 'attacker_acc', 'defender_acc', 'type1_distance', 'win_time(s)', 'num_of_attack']]
player_stats1.fillna(0, inplace=True)

In [73]:
player_stats1

,created_at,player,time_survived(s),kill,attacker_kill,defender_kill,damage,attacker_damage,defender_damage,win_rate,...,defender_win_rate,survival_rate,attacker_survival_rate,defender_survival_rate,acc,attacker_acc,defender_acc,type1_distance,win_time(s),num_of_attack
0,2020-09-25T10:28:20Z,VSG_Foxy,1263.289,0.0,0.0,0.0,186.056967,186.056967,0.000000,0.000000,...,0.0,0.800000,0.5,1.000000,0.092571,0.160000,0.047619,0.000000,9999.0,2.0
1,2020-09-25T10:28:20Z,VSG_DG98,1098.276,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.500000,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,9999.0,0.0
2,2020-09-25T10:28:20Z,VSG_Menteul,1263.279,2.0,1.0,1.0,111.782974,0.000000,111.782974,0.666667,...,0.5,0.800000,1.0,0.750000,0.300000,0.500000,0.250000,1512.380409,8.5,1.0
3,2020-09-25T10:28:20Z,VSG_Hulk,1236.989,1.0,1.0,0.0,185.693186,185.693186,0.000000,0.500000,...,0.0,0.800000,1.0,0.666667,0.078325,0.195813,0.000000,6971.851654,3.0,2.0
4,2020-09-25T10:28:20Z,GEN_Loki,1655.140,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.500000,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,9999.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2020-09-26T11:42:20Z,VLG_DAEVA,1700.912,1.0,1.0,0.0,297.907040,134.706532,163.200508,0.500000,...,0.0,0.857143,1.0,0.750000,0.116772,0.257143,0.011494,17597.325612,4.0,3.0
2556,2020-09-26T11:42:20Z,VSG_Hulk,1578.199,0.0,0.0,0.0,191.487841,191.487841,0.000000,0.000000,...,0.0,1.000000,1.0,1.000000,0.044714,0.055892,0.000000,0.000000,9999.0,4.0
2557,2020-09-26T11:42:20Z,VSG_DG98,1578.188,4.0,4.0,0.0,179.347429,179.347429,0.000000,0.800000,...,0.0,0.875000,1.0,0.666667,0.294173,0.470677,0.000000,21704.925898,13.0,5.0
2558,2020-09-26T11:42:20Z,VSG_Menteul,1393.822,0.0,0.0,0.0,142.227491,142.227491,0.000000,0.000000,...,0.0,0.800000,1.0,0.500000,0.237003,0.395005,0.000000,0.000000,9999.0,3.0


In [93]:
mean_df = player_stats1.groupby('player').mean()
mean_df

,time_survived(s),kill,attacker_kill,defender_kill,damage,attacker_damage,defender_damage,win_rate,attacker_win_rate,defender_win_rate,survival_rate,attacker_survival_rate,defender_survival_rate,acc,attacker_acc,defender_acc,type1_distance,win_time(s),num_of_attack
player,,,,,,,,,,,,,,,,,,,
AFK_APXB,1148.701600,0.800000,0.700000,0.100000,124.212777,83.163274,41.049503,0.330000,0.450000,0.050000,0.701667,0.800000,0.591667,0.241257,0.349348,0.109562,4679.071794,5014.900000,1.500000
AFK_Arden,1128.020100,1.000000,1.000000,0.000000,147.705956,141.185456,6.520500,0.408333,0.516667,0.000000,0.723333,0.630000,0.416667,0.212532,0.314024,0.021429,3965.908764,4003.500000,2.000000
AFK_Asag2,1075.895200,0.600000,0.500000,0.100000,101.019361,75.923203,25.096158,0.150000,0.200000,0.050000,0.542500,0.500000,0.366667,0.191188,0.267172,0.057901,1839.285648,8003.700000,1.500000
AFK_Caley,1082.157800,0.200000,0.200000,0.000000,44.061865,36.850098,7.211768,0.100000,0.200000,0.000000,0.481667,0.500000,0.404167,0.179048,0.322857,0.133333,1201.466944,7999.200000,0.600000
AFS_BBABBO,1241.002200,1.700000,1.600000,0.100000,294.176603,272.392602,21.784001,0.466667,0.625000,0.100000,0.701429,0.658333,0.500000,0.305330,0.294098,0.104494,6557.651524,3034.333333,3.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VSG_WICK2D,1188.877800,0.900000,0.800000,0.100000,123.819470,103.300107,20.519363,0.308333,0.400000,0.050000,0.682381,0.700000,0.491667,0.174863,0.401182,0.015341,4025.930420,5047.583333,1.600000
emT_Gyuyeon,1315.840300,1.166667,1.000000,0.166667,170.088980,137.172715,32.916265,0.343333,0.433333,0.100000,0.678082,0.733333,0.505556,0.226617,0.336862,0.115422,4846.249245,5364.207778,2.266667
emT_Hwarang,1286.871433,1.000000,0.833333,0.166667,174.493795,154.253643,20.240152,0.330556,0.505556,0.050000,0.663479,0.818333,0.416667,0.277396,0.392028,0.071030,6790.508883,4352.179167,2.400000


In [98]:
mean_df.query("attacker_acc<defender_acc")

,time_survived(s),kill,attacker_kill,defender_kill,damage,attacker_damage,defender_damage,win_rate,attacker_win_rate,defender_win_rate,survival_rate,attacker_survival_rate,defender_survival_rate,acc,attacker_acc,defender_acc,type1_distance,win_time(s),num_of_attack
player,,,,,,,,,,,,,,,,,,,
GRF_Feint,1173.5624,0.4,0.4,0.0,62.295513,35.726224,26.569289,0.133333,0.2,0.0,0.538095,0.2,0.516667,0.181746,0.112963,0.133333,695.958651,8000.0,0.6


## 5. MANOVA

In [88]:
attacker_stats = player_stats1[['attacker_kill', 'attacker_damage', 'attacker_win_rate', 'attacker_acc', 'attacker_survival_rate']]
attacker_stats['class'] = 0
attacker_stats.columns = ['kill', 'damage', 'win_rate', 'acc', 'survival_rate', 'target']
defender_stats = player_stats1[['defender_kill', 'defender_damage', 'defender_win_rate', 'defender_acc', 'defender_survival_rate']]
defender_stats['class'] = 1
defender_stats.columns = ['kill', 'damage', 'win_rate', 'acc', 'survival_rate', 'target']

In [89]:
data = pd.concat([attacker_stats, defender_stats])
data

,kill,damage,win_rate,acc,survival_rate,target
0,0.0,186.056967,0.0,0.160000,0.500000,0
1,0.0,0.000000,0.0,0.000000,0.000000,0
2,1.0,0.000000,1.0,0.500000,1.000000,0
3,1.0,185.693186,1.0,0.195813,1.000000,0
4,0.0,0.000000,0.0,0.000000,0.000000,0
...,...,...,...,...,...,...
2555,0.0,163.200508,0.0,0.011494,0.750000,1
2556,0.0,0.000000,0.0,0.000000,1.000000,1
2557,0.0,0.000000,0.0,0.000000,0.666667,1
2558,0.0,0.000000,0.0,0.000000,0.500000,1


In [91]:
from statsmodels.multivariate.manova import MANOVA
print(MANOVA.from_formula("kill+damage+win_rate+acc+survival_rate~C(target)", data=data).mv_test())

                   Multivariate linear model
                                                                
----------------------------------------------------------------
       Intercept        Value  Num DF   Den DF   F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda 0.3807 5.0000 5114.0000 1663.6259 0.0000
         Pillai's trace 0.6193 5.0000 5114.0000 1663.6259 0.0000
 Hotelling-Lawley trace 1.6265 5.0000 5114.0000 1663.6259 0.0000
    Roy's greatest root 1.6265 5.0000 5114.0000 1663.6259 0.0000
----------------------------------------------------------------
                                                                
----------------------------------------------------------------
        C(target)        Value  Num DF   Den DF  F Value  Pr > F
----------------------------------------------------------------
           Wilks' lambda 0.6735 5.0000 5114.0000 495.8896 0.0000
          Pillai's trace 0.3265 5.0000 5114.0